In [1]:
%cd ..

/home/jovyan/ice-cube


In [2]:
import math
import os

import pandas as pd
import torch

In [3]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [4]:
##### INSERT SOURCE CODE HERE FOR SUBMISSION #####

In [5]:
from omegaconf import OmegaConf

omega_conf_dir = "./config"

c_main = OmegaConf.load(os.path.join(omega_conf_dir, "main.yaml"))
c_data = OmegaConf.load(os.path.join(omega_conf_dir, "data.yaml"))
c = OmegaConf.merge(c_main, c_data)

c.wandb.enabled = False

In [6]:
from src.ice_cube.data_loader import (
    make_test_dataloader,
    make_test_dataloader_plus_minus,
    make_train_dataloader,
)
from src.ice_cube.model import load_pretrained_model
from src.ice_cube.scoring import angular_dist_score
from src.ice_cube.submission import to_submission_df

graphnet: INFO     2023-02-24 06:48:57 - get_logger - Writing log to logs/graphnet_20230224-064857.log


2023-02-24 06:48:57,234 [INFO] [logging] Writing log to logs/graphnet_20230224-064857.log


graphnet: WARNING  2023-02-24 06:48:57 - warn_once - `icecube` not available. Some functionality may be missing.


2023-02-24 06:48:57,241 [WARNING] [logging] `icecube` not available. Some functionality may be missing.
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


## SQLiteDataset

In [7]:
train_loader, valid_loader = make_train_dataloader(c)

In [8]:
test_loader = make_test_dataloader(c)

graphnet: WARNING  2023-02-24 06:48:59 - SQLiteDataset.warning - Removing the following (missing) truth variables: zenith, azimuth


2023-02-24 06:48:59,908 [WARNING] [logging] Removing the following (missing) truth variables: zenith, azimuth


In [9]:
test_pm_loader = make_test_dataloader_plus_minus(c)

graphnet: WARNING  2023-02-24 06:48:59 - SQLiteDataset.warning - Removing the following (missing) truth variables: zenith, azimuth


2023-02-24 06:48:59,917 [WARNING] [logging] Removing the following (missing) truth variables: zenith, azimuth


## Valid Data

In [10]:
type(valid_loader.dataset)

graphnet.data.sqlite.sqlite_dataset.SQLiteDataset

In [11]:
train_label = train_loader.dataset.query_table(
    "meta_table", ["event_id", "azimuth", "zenith"]
)

In [12]:
valid_label = valid_loader.dataset.query_table(
    "meta_table", ["event_id", "azimuth", "zenith"]
)

In [13]:
train_df = pd.DataFrame(
    train_label, columns=["event_id", "azimuth", "zenith"]
).set_index("event_id")

In [14]:
valid_df = pd.DataFrame(
    valid_label, columns=["event_id", "azimuth", "zenith"]
).set_index("event_id")

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 24 to 3266196
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   azimuth  200000 non-null  float64
 1   zenith   200000 non-null  float64
dtypes: float64(2)
memory usage: 4.6 MB


In [16]:
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 24 to 3266196
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   azimuth  200000 non-null  float64
 1   zenith   200000 non-null  float64
dtypes: float64(2)
memory usage: 4.6 MB


In [17]:
# model = load_pretrained_model(c, valid_loader)

In [18]:
# results = model.predict_as_dataframe(
#     gpus=[0],
#     dataloader=valid_loader,
#     prediction_columns=model.prediction_columns,
#     additional_attributes=[c.settings.index_name],
# )

In [19]:
# results.info()

In [20]:
# submission_df = to_submission_df(results)

In [21]:
# submission_df.info()

In [22]:
# part_valid_df = valid_df.loc[submission_df.index, :]

In [23]:
# part_valid_df.info()

In [24]:
# score = angular_dist_score(
#     part_valid_df["azimuth"],
#     part_valid_df["zenith"],
#     submission_df["azimuth"],
#     submission_df["zenith"],
# )

In [25]:
# score

## Test Data

In [26]:
model = load_pretrained_model(c, test_loader)

2023-02-24 06:49:00,437 [INFO] [instantiator] Created a temporary directory at /tmp/tmpkq7rt1ey
2023-02-24 06:49:00,438 [INFO] [instantiator] Writing /tmp/tmpkq7rt1ey/_remote_module_non_sriptable.py


In [27]:
results = model.predict_as_dataframe(
    gpus=[0],
    dataloader=test_loader,
    prediction_columns=model.prediction_columns,
    additional_attributes=[c.settings.index_name],
)

2023-02-24 06:49:00,527 [INFO] [rank_zero] GPU available: True (cuda), used: True
2023-02-24 06:49:00,529 [INFO] [rank_zero] TPU available: False, using: 0 TPU cores
2023-02-24 06:49:00,529 [INFO] [rank_zero] IPU available: False, using: 0 IPUs
2023-02-24 06:49:00,530 [INFO] [rank_zero] HPU available: False, using: 0 HPUs
2023-02-24 06:49:00,533 [INFO] [rank_zero] GPU available: True (cuda), used: True
2023-02-24 06:49:00,533 [INFO] [rank_zero] TPU available: False, using: 0 TPU cores
2023-02-24 06:49:00,533 [INFO] [rank_zero] IPU available: False, using: 0 IPUs
2023-02-24 06:49:00,534 [INFO] [rank_zero] HPU available: False, using: 0 HPUs
2023-02-24 06:49:02,885 [INFO] [cuda] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [28]:
results

,direction_x,direction_y,direction_z,direction_kappa,event_id
0,0.987893,-0.148705,0.044195,2.946703,2092.0
1,-0.551074,-0.254085,-0.794832,6.980618,7344.0
2,-0.164367,-0.986232,0.018184,86.766510,9482.0


In [29]:
submission_df = to_submission_df(results)

In [30]:
submission_df

,azimuth,zenith
event_id,,
2092.0,6.133780,1.526587
7344.0,3.573616,2.489527
9482.0,4.547245,1.552611


In [31]:
model = load_pretrained_model(c, test_pm_loader)

In [32]:
results = model.predict_as_dataframe(
    gpus=[0],
    dataloader=test_pm_loader,
    prediction_columns=model.prediction_columns,
    additional_attributes=[c.settings.index_name],
)

2023-02-24 06:49:03,777 [INFO] [rank_zero] GPU available: True (cuda), used: True
2023-02-24 06:49:03,778 [INFO] [rank_zero] TPU available: False, using: 0 TPU cores
2023-02-24 06:49:03,779 [INFO] [rank_zero] IPU available: False, using: 0 IPUs
2023-02-24 06:49:03,779 [INFO] [rank_zero] HPU available: False, using: 0 HPUs
2023-02-24 06:49:03,782 [INFO] [rank_zero] GPU available: True (cuda), used: True
2023-02-24 06:49:03,783 [INFO] [rank_zero] TPU available: False, using: 0 TPU cores
2023-02-24 06:49:03,784 [INFO] [rank_zero] IPU available: False, using: 0 IPUs
2023-02-24 06:49:03,784 [INFO] [rank_zero] HPU available: False, using: 0 HPUs
2023-02-24 06:49:03,790 [INFO] [cuda] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [33]:
results

,direction_x,direction_y,direction_z,direction_kappa,event_id
0,-0.717383,0.607155,0.341649,11.453881,2092.0
1,0.494068,0.071199,0.866503,0.832128,7344.0
2,0.098816,0.994516,-0.034239,83.222397,9482.0


In [40]:
results["direction_x"] = -results["direction_x"]
results["direction_y"] = -results["direction_y"]
results["direction_z"] = -results["direction_z"]

In [41]:
submission_tta_df = to_submission_df(results)

In [42]:
submission_tta_df

,azimuth,zenith
event_id,,
2092.0,5.580815,1.919467
7344.0,3.284714,2.618950
9482.0,4.613353,1.536550


In [44]:
score = angular_dist_score(
    submission_df["azimuth"],
    submission_df["zenith"],
    submission_tta_df["azimuth"],
    submission_tta_df["zenith"],
)
score

0.31426566129194694

In [47]:
score * 180 / math.pi

18.0060960379164

In [36]:
for i in test_loader:
    log.info(i.get_example(1).features)
    log.info(i.get_example(1).x.shape)
    log.info(i.get_example(1).x[0])
    log.info(i.get_example(1).y[0])
    log.info(i.get_example(1).z[0])
    log.info(i.get_example(1).time[0])
    log.info(i.get_example(1).charge[0])
    # log.info(i.get_example(1).to_namedtuple())
    break

2023-02-24 06:49:04,308 [INFO] [3122733060] ['x', 'y', 'z', 'time', 'charge', 'auxiliary']
2023-02-24 06:49:04,309 [INFO] [3122733060] torch.Size([36, 6])
2023-02-24 06:49:04,309 [INFO] [3122733060] tensor([-1.0970e+01,  6.7200e+00, -4.2333e+02,  5.9290e+03,  1.0250e+00,
         1.0000e+00])
2023-02-24 06:49:04,311 [INFO] [3122733060] tensor(6.7200)
2023-02-24 06:49:04,312 [INFO] [3122733060] tensor(-423.3300)
2023-02-24 06:49:04,313 [INFO] [3122733060] tensor(5929.)
2023-02-24 06:49:04,314 [INFO] [3122733060] tensor(1.0250)


In [37]:
for i in test_pm_loader:
    log.info(i.get_example(1).features)
    log.info(i.get_example(1).x.shape)
    log.info(i.get_example(1).x[0])
    log.info(i.get_example(1).y[0])
    log.info(i.get_example(1).z[0])
    log.info(i.get_example(1).time[0])
    log.info(i.get_example(1).charge[0])
    break

2023-02-24 06:49:04,342 [INFO] [1029255878] ['x', 'y', 'z', 'time', 'charge', 'auxiliary']
2023-02-24 06:49:04,343 [INFO] [1029255878] torch.Size([36, 6])
2023-02-24 06:49:04,344 [INFO] [1029255878] tensor([ 1.0970e+01, -6.7200e+00,  4.2333e+02,  5.9290e+03,  1.0250e+00,
         1.0000e+00])
2023-02-24 06:49:04,345 [INFO] [1029255878] tensor(-6.7200)
2023-02-24 06:49:04,346 [INFO] [1029255878] tensor(423.3300)
2023-02-24 06:49:04,346 [INFO] [1029255878] tensor(5929.)
2023-02-24 06:49:04,347 [INFO] [1029255878] tensor(1.0250)
